## 1. The new dataset class for Subject Dependent

The `data` is loaded during the `.get_data*()` call. It will select the label according to the ENUM `Dataset_subjectDependent.STIMULI_AROUSAL` and `Dataset_subjectDependent.STIMULI_VALENCE`.

The `label` is convert to `{0,1}` base on the mean of that selected data.

The `data` and `label` are then split with `test_size = 0.3` and `shuffle` in the split function followed by segmentation (according to the .set_segment() number. Default:1)

In [24]:
from components.dataset_jo import *
import time
from tqdm.notebook import tqdm

def train_model(X_ori,y_ori, kernel='rbf'):
    # Make a copy because I am paranoid
    X,y = X_ori.copy(), y_ori.copy()

    from sklearn.svm import SVC
    from sklearn.utils import shuffle
    from sklearn.model_selection import cross_val_score

    X_shuff,y_shuff = shuffle(X,y)
    model = SVC(kernel=kernel,max_iter=50000)
    cross = cross_val_score(model, X_shuff, y_shuff, cv=10)

    model = SVC(kernel=kernel, max_iter=50000)
    model.fit(X_shuff, y_shuff)
    ans = model.predict(X_shuff)
    acc = sum(ans == y_shuff) / len(y_shuff)
    return model, acc, cross

dataset = Dataset_subjectDependent('data')
dataset.set_segment(60)

Found: 32 files


In [ ]:
# You can get the data in to way
# First way, the numpy array
data_train, data_test, labels_train, labels_test = dataset.get_data_numpy('s01', Dataset_subjectDependent.STIMULI_AROUSAL)
print(data_train.shape, data_test.shape, labels_train.shape, labels_test.shape)

# Second way, as pytorch.util.Dataset
train_dataset, test_dataset = dataset.get_data_torch_dataset('s01', Dataset_subjectDependent.STIMULI_VALENCE)
print(len(train_dataset), len(test_dataset) )
# This train_dataset, and test_dataset is DataLoader ready
# train_loader = torch.utils.data.DataLoader(train_dataset, batch=8)

## 2. Spectral Features

In [3]:
from mne_features.feature_extraction import FeatureExtractor

In [13]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})
fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)

accs,cv_means,cv_stds  = [],[],[]
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1))
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

accs_v,cv_means_v,cv_stds_v  = [],[],[]
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1))
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.76917|10-CV=0.6875|STD=0.02582|Time spend=4.8965325355529785
	AROUSAL-s02|Acc=0.67292|10-CV=0.63875|STD=0.02387|Time spend=5.065949201583862
	AROUSAL-s03|Acc=0.69875|10-CV=0.6325|STD=0.02435|Time spend=5.33520245552063
	AROUSAL-s04|Acc=0.6775|10-CV=0.62875|STD=0.0408|Time spend=4.705340147018433
	AROUSAL-s05|Acc=0.62875|10-CV=0.60375|STD=0.00976|Time spend=5.062718152999878
	AROUSAL-s06|Acc=0.63292|10-CV=0.59958|STD=0.02524|Time spend=4.962883472442627
	AROUSAL-s07|Acc=0.74167|10-CV=0.65833|STD=0.04187|Time spend=5.100658178329468
	AROUSAL-s08|Acc=0.71083|10-CV=0.63292|STD=0.02972|Time spend=4.9638588428497314
	AROUSAL-s09|Acc=0.71458|10-CV=0.65708|STD=0.01729|Time spend=5.046931505203247
	AROUSAL-s10|Acc=0.75208|10-CV=0.70542|STD=0.02958|Time spend=4.747288465499878
	AROUSAL-s11|Acc=0.64083|10-CV=0.61417|STD=0.0293|Time spend=4.990945339202881
	AROUSAL-s12|Acc=0.66708|10-CV=0.65167|STD=0.02623|Time spend=4.959524869918823
	AROUSAL-s13|Acc=0.64958|10-CV=0.61833|STD=0

## 3. Asymetry

In [14]:
channels = ['Fp1','AF3','F3','F7','FC5','FC1','C3','T7','CP5','CP1','P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4','F8','FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
left_channels = ['Fp1','F7','F3','T7','P7','C3','P3','O1','AF3','FC5','FC1','CP5','CP1','PO3']
right_channels = ['Fp2','F8','F4','T8','P8','C4','P4','O2','AF4','FC6','FC2','CP6','CP2','PO4']
left_channel_indexes = [ channels.index(ch) for ch in left_channels ]
right_channel_indexes = [ channels.index(ch) for ch in right_channels ]

print(f"{left_channel_indexes=}")
print(f"{right_channel_indexes=}")

frontal_channels = ['FC5','FC1','FC2','FC6','F7','F3','Fz','F4','F8','Fp1','Fp2']
posterior_channels = ['CP5','CP1','CP2','CP6','P7','P3','Pz','P4','P8','O1','O2']

frontal_channel_indexes = [ channels.index(ch) for ch in frontal_channels ]
posterior_channel_indexes = [ channels.index(ch) for ch in posterior_channels ]

print(f"{frontal_channel_indexes=}")
print(f"{posterior_channel_indexes=}")



left_channel_indexes=[0, 3, 2, 7, 11, 6, 10, 13, 1, 4, 5, 8, 9, 12]
right_channel_indexes=[16, 20, 19, 25, 29, 24, 28, 31, 17, 21, 22, 26, 27, 30]
frontal_channel_indexes=[4, 5, 22, 21, 3, 2, 18, 19, 20, 0, 16]
posterior_channel_indexes=[8, 9, 27, 26, 11, 10, 15, 28, 29, 13, 31]


In [27]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

accs,cv_means,cv_stds  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1))
        print(f"\tAROUSAL-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs[feature_name].append(acc)
        cv_means[feature_name].append(cross.mean())
        cv_stds[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"AROUSAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

accs_v,cv_means_v,cv_stds_v  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1))
        print(f"\tVALENCE-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs_v[feature_name].append(acc)
        cv_means_v[feature_name].append(cross.mean())
        cv_stds_v[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"VALENCE-{feature_name}|Acc={sum(accs_v[feature_name])/len(accs_v[feature_name])}|10-CV={sum(cv_means_v[feature_name])/len(cv_means_v[feature_name])}|STD={sum(cv_stds_v[feature_name])/len(cv_stds_v[feature_name])}")

for feature_name in ['DASM','RASM','DCAU']:
    accs[feature_name].extend(accs_v[feature_name])
    cv_means[feature_name].extend(cv_means_v[feature_name])
    cv_stds[feature_name].extend(cv_stds_v[feature_name])
    print(f"TOTAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

	AROUSAL-s01-DASM|Acc=0.61|10-CV=0.60125|STD=0.00591|Time spend=3.4438962936401367
	AROUSAL-s01-RASM|Acc=0.60833|10-CV=0.60167|STD=0.00276|Time spend=6.116313219070435
	AROUSAL-s01-DCAU|Acc=0.63167|10-CV=0.61458|STD=0.01551|Time spend=8.154110670089722
	AROUSAL-s02-DASM|Acc=0.61|10-CV=0.57333|STD=0.02329|Time spend=3.3877434730529785
	AROUSAL-s02-RASM|Acc=0.53583|10-CV=0.52625|STD=0.00834|Time spend=5.936612844467163
	AROUSAL-s02-DCAU|Acc=0.62208|10-CV=0.58708|STD=0.03844|Time spend=7.928848505020142
	AROUSAL-s03-DASM|Acc=0.59167|10-CV=0.5475|STD=0.02473|Time spend=3.299607992172241
	AROUSAL-s03-RASM|Acc=0.5375|10-CV=0.52917|STD=0.00968|Time spend=5.6853928565979
	AROUSAL-s03-DCAU|Acc=0.57875|10-CV=0.535|STD=0.02858|Time spend=7.843799829483032
	AROUSAL-s04-DASM|Acc=0.6325|10-CV=0.60917|STD=0.02794|Time spend=2.923250198364258
	AROUSAL-s04-RASM|Acc=0.55375|10-CV=0.5475|STD=0.00333|Time spend=5.007073640823364
	AROUSAL-s04-DCAU|Acc=0.61333|10-CV=0.59083|STD=0.02998|Time spend=6.83664917

## 4. Connectivity

### 4.1 coherence

In [6]:
from mne_connectivity import spectral_connectivity_time

def save(data,filename):
    with open(f'cache/{filename}.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
    with open(f'cache/{filename}.pickle', 'rb') as handle:
        data = pickle.load(handle)
    return data

In [4]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
for filename in tqdm(dataset.get_file_list()):
    # if(filename in ['s01','s02','s03','s04','s05','s06','s07']): continue
    print("----"*5, filename, "----"*5)
    # start = time.time()
    epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in range(0,len(epochs), step):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='coh',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoch, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_COH_AROUSAL_{filename}")


    epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in tqdm(range(0,len(epochs), step)):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='coh',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoc
        # h, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_COH_VALENCE_{filename}")

In [13]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_COH_AROUSAL_{filename}")
    assert X.shape == (2400,2112)
    _, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    _, acc, cross = train_model(X, labels.reshape(-1))
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

  0%|          | 0/32 [00:00<?, ?it/s]

	AROUSAL-s01|Acc=0.77917|10-CV=0.68208|STD=0.02084|Time spend=64.93514370918274
	AROUSAL-s02|Acc=0.70125|10-CV=0.66167|STD=0.02998|Time spend=66.43738794326782
	AROUSAL-s03|Acc=0.7425|10-CV=0.63583|STD=0.03035|Time spend=69.64683055877686
	AROUSAL-s04|Acc=0.735|10-CV=0.66708|STD=0.02792|Time spend=64.85696578025818
	AROUSAL-s05|Acc=0.6975|10-CV=0.64583|STD=0.01667|Time spend=71.05305814743042
	AROUSAL-s06|Acc=0.67792|10-CV=0.625|STD=0.02041|Time spend=69.97903823852539
	AROUSAL-s07|Acc=0.78125|10-CV=0.67625|STD=0.03085|Time spend=66.84944558143616
	AROUSAL-s08|Acc=0.74167|10-CV=0.63875|STD=0.03385|Time spend=65.9965980052948
	AROUSAL-s09|Acc=0.77625|10-CV=0.70125|STD=0.03416|Time spend=67.61231112480164
	AROUSAL-s10|Acc=0.80333|10-CV=0.75|STD=0.03107|Time spend=60.44512343406677
	AROUSAL-s11|Acc=0.69125|10-CV=0.65083|STD=0.02348|Time spend=69.42803263664246
	AROUSAL-s12|Acc=0.71333|10-CV=0.66667|STD=0.02197|Time spend=64.91551637649536
	AROUSAL-s13|Acc=0.67542|10-CV=0.65042|STD=0.01125

In [14]:
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

AROUSAL|Acc=0.7279817708333334|10-CV=0.6657552083333331|STD=0.020983067047204703


In [16]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_COH_VALENCE_{filename}")
    assert X.shape == (2400,2112)
    _, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    _, acc, cross = train_model(X, labels.reshape(-1))
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	VALENCE-s01|Acc=0.72083|10-CV=0.5975|STD=0.01837|Time spend=69.1800467967987
	VALENCE-s02|Acc=0.75833|10-CV=0.70125|STD=0.02075|Time spend=60.175132274627686
	VALENCE-s03|Acc=0.81792|10-CV=0.75917|STD=0.03475|Time spend=52.87031078338623
	VALENCE-s04|Acc=0.75792|10-CV=0.69625|STD=0.02578|Time spend=58.85808539390564
	VALENCE-s05|Acc=0.72542|10-CV=0.64833|STD=0.0293|Time spend=67.06726145744324
	VALENCE-s06|Acc=0.72958|10-CV=0.65542|STD=0.01571|Time spend=65.33635210990906
	VALENCE-s07|Acc=0.79|10-CV=0.7125|STD=0.02394|Time spend=56.17925047874451
	VALENCE-s08|Acc=0.74917|10-CV=0.69958|STD=0.01976|Time spend=62.60816264152527
	VALENCE-s09|Acc=0.77417|10-CV=0.72375|STD=0.02472|Time spend=59.522428035736084
	VALENCE-s10|Acc=0.85083|10-CV=0.81833|STD=0.02839|Time spend=45.94798946380615
	VALENCE-s11|Acc=0.71083|10-CV=0.67542|STD=0.02585|Time spend=65.4010488986969
	VALENCE-s12|Acc=0.75958|10-CV=0.6625|STD=0.0337|Time spend=63.1700873374939
	VALENCE-s13|Acc=0.7675|10-CV=0.73083|STD=0.02167

In [17]:
accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

TOTAL|Acc=0.7413151041666667|10-CV=0.6775585937499997|STD=0.02208682347592504


### Connectivity PLV

In [ ]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
for filename in tqdm(dataset.get_file_list()):
    # if(filename in ['s01','s02','s03','s04','s05','s06','s07']): continue
    print("----"*5, filename, "----"*5)
    # start = time.time()
    epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in range(0,len(epochs), step):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='plv',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoch, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_PLV_AROUSAL_{filename}")


    epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in tqdm(range(0,len(epochs), step)):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='plv',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoc
        # h, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_PLV_VALENCE_{filename}")

In [21]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_PLV_AROUSAL_{filename}")
    assert X.shape == (2400,2112)
    _, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    _, acc, cross = train_model(X, labels.reshape(-1))
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	AROUSAL-s01|Acc=0.75542|10-CV=0.66833|STD=0.02291|Time spend=63.03299117088318
	AROUSAL-s02|Acc=0.70708|10-CV=0.66708|STD=0.02375|Time spend=61.41347575187683
	AROUSAL-s03|Acc=0.74083|10-CV=0.62417|STD=0.03455|Time spend=66.05428910255432
	AROUSAL-s04|Acc=0.73542|10-CV=0.65958|STD=0.02197|Time spend=58.56443166732788
	AROUSAL-s05|Acc=0.71708|10-CV=0.64917|STD=0.02031|Time spend=64.23791289329529
	AROUSAL-s06|Acc=0.675|10-CV=0.61625|STD=0.01932|Time spend=63.78882026672363
	AROUSAL-s07|Acc=0.77417|10-CV=0.66167|STD=0.03327|Time spend=63.45789623260498
	AROUSAL-s08|Acc=0.73667|10-CV=0.64625|STD=0.01671|Time spend=63.11403250694275
	AROUSAL-s09|Acc=0.77458|10-CV=0.7025|STD=0.01856|Time spend=63.68644738197327
	AROUSAL-s10|Acc=0.79917|10-CV=0.74458|STD=0.02305|Time spend=59.18967938423157
	AROUSAL-s11|Acc=0.69167|10-CV=0.65042|STD=0.02087|Time spend=66.4958643913269
	AROUSAL-s12|Acc=0.71208|10-CV=0.66042|STD=0.0177|Time spend=62.903270959854126
	AROUSAL-s13|Acc=0.67167|10-CV=0.65125|STD=0

In [22]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_PLV_VALENCE_{filename}")
    assert X.shape == (2400,2112)
    _, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    _, acc, cross = train_model(X, labels.reshape(-1))
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)

print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	VALENCE-s01|Acc=0.715|10-CV=0.59583|STD=0.01981|Time spend=117.21722841262817
	VALENCE-s02|Acc=0.76375|10-CV=0.70292|STD=0.02733|Time spend=118.19430232048035
	VALENCE-s03|Acc=0.815|10-CV=0.74583|STD=0.02472|Time spend=109.09722566604614
	VALENCE-s04|Acc=0.76208|10-CV=0.69042|STD=0.02313|Time spend=110.54049324989319
	VALENCE-s05|Acc=0.73417|10-CV=0.64125|STD=0.02382|Time spend=119.80090141296387
	VALENCE-s06|Acc=0.72667|10-CV=0.64792|STD=0.01992|Time spend=81.20034623146057
	VALENCE-s07|Acc=0.78333|10-CV=0.7075|STD=0.01161|Time spend=57.46352481842041
	VALENCE-s08|Acc=0.75458|10-CV=0.69917|STD=0.01997|Time spend=65.95078611373901
	VALENCE-s09|Acc=0.78125|10-CV=0.71917|STD=0.02388|Time spend=59.49257016181946
	VALENCE-s10|Acc=0.84917|10-CV=0.8125|STD=0.02958|Time spend=44.80204176902771
	VALENCE-s11|Acc=0.71|10-CV=0.665|STD=0.0304|Time spend=64.8858253955841
	VALENCE-s12|Acc=0.76083|10-CV=0.66667|STD=0.0107|Time spend=61.64257526397705
	VALENCE-s13|Acc=0.76375|10-CV=0.72042|STD=0.0244

## 5. CSP

In [30]:
from components.CSP import CSP
from mne_features.feature_extraction import FeatureExtractor

bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

In [35]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, label = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    label = label.squeeze()
    start = time.time()
    filters = CSP(data[label==0], data[label==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[label[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape == (2400,160)
    _, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    _, acc, cross = train_model(X, labels.reshape(-1))
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	AROUSAL-s01|Acc=0.575|10-CV=0.56833|STD=0.02597|Time spend=54.61695194244385


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s02|Acc=0.63458|10-CV=0.62125|STD=0.03187|Time spend=6.6895201206207275


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


...

	AROUSAL-s30|Acc=0.66083|10-CV=0.64125|STD=0.02729|Time spend=5.268497467041016


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s31|Acc=0.66333|10-CV=0.64083|STD=0.03744|Time spend=5.89745569229126


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s32|Acc=0.59292|10-CV=0.57542|STD=0.02544|Time spend=5.9968421459198
AROUSAL|Acc=0.6228125000000002|10-CV=0.60984375|STD=0.02216186258106818


In [36]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, label = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    label = label.squeeze()
    start = time.time()
    filters = CSP(data[label==0], data[label==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[label[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape == (2400,160)
    _, labels = dataset.get_data_numpy(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, split=False)
    _, acc, cross = train_model(X, labels.reshape(-1))
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s01|Acc=0.70708|10-CV=0.69708|STD=0.02814|Time spend=5.238353967666626


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s02|Acc=0.99708|10-CV=0.99667|STD=0.00363|Time spend=1.6159617900848389


...

	VALENCE-s30|Acc=0.87625|10-CV=0.87125|STD=0.01061|Time spend=3.785003423690796


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s31|Acc=0.555|10-CV=0.54708|STD=0.00747|Time spend=6.546914100646973


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s32|Acc=0.57417|10-CV=0.555|STD=0.02892|Time spend=6.225713729858398
VALENCE|Acc=0.7662239583333333|10-CV=0.7571484375|STD=0.01819082529684967


In [37]:
accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)

print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

TOTAL|Acc=0.6945182291666667|10-CV=0.6834960937500001|STD=0.020176343938958924
